# Imports

In [2]:
import pysweep
import pysweep.databackends.debug
import pysweep.databackends.list_backend
import pysweep.databackends.qcodes
import pysweep.core.measurementfunctions
from pysweep.core.sweepobject import SweepObject
from pysweep.core.measurementfunctions import MakeMeasurementFunction
from pysweep.databackends.base import DataParameterFixedAxis
from pysweep.databackends.base import DataParameter
import pysweep.convenience as conv

from pytopo.qctools.dataset2 import select_experiment

import qcodes as qc
from qcodes.dataset.measurements import Measurement
from qcodes.utils.validators import Ints, Numbers
from qcodes.math.field_vector import FieldVector
import qcodes.dataset.plotting

from time import sleep
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\qcodes\dataset\sqlite_base.py:51: UserWarning: The module `qcodes.dataset.sqlite_base` is deprecated.
Public features are available at the import of `qcodes`.
Private features are available in `qcodes.dataset.sqlite.*` modules.
  warnings.warn('The module `qcodes.dataset.sqlite_base` is deprecated.\n'


In [ ]:
def mirror_measurement_dir_to_data_dir(db_name='experiments.db'):
    from pathlib import Path
    db_directory = Path('D:/Data')
    curr_path = Path.cwd()
    parts_curr_path = curr_path.parts       
    dest_dir = Path(db_directory, *parts_curr_path[-2:])
    dest_dir = Path(str(dest_dir).replace(' ', '_'))
    if not dest_dir.exists():
        dest_dir.mkdir(parents=True)
    dest_path = Path(dest_dir, db_name)
    qc.initialise_or_create_database_at(dest_path)
    qc.config['core']['db_location'] = dest_path

In [ ]:
def create_local_dbase_in(folder_name='general', db_name='experiments.db'):
    """    
    Initialise or create a QCoDeS database in D:/Data/folder_name/db_name 
    If the directory does not exist, it is created.
    
    Inputs:
    folder_name (str): Name of the subfolder in D:/Data where db is stored. 
                       Can also be a path to a subfolder, e.g. general/sample1 leads to db in D:/Data/general/sample1 
    db_name (str):     Name of database, including .db
    """
    from pathlib import Path
    
    dest_dir = Path(r'D:/Data', folder_name)
    if not dest_dir.exists():
        dest_dir.mkdir(parents=True)        
    dest_path = Path(dest_dir, db_name)
    qc.initialise_or_create_database_at(dest_path)
    qc.config['core']['db_location'] = dest_path

# Setup of database and station

## Set up measurement station

for now, we keep the initalisation file <br>
**TODO:** move station import to YAML file  <br>
https://qcodes.github.io/Qcodes/examples/Station.html#Using-Station-with-a-YAML-configuration-file

### K1

### K2

In [ ]:
%run -i "D:\OneDrive\Setups\LK2\LK2_code\Code\LK2\LK2\init_station_VNA.py"

In [ ]:
from qcodes_contrib_drivers.drivers.QuTech.IVVI import IVVI
ivvi = create_inst(IVVI, 'ivvi', address='ASRL4', numdacs=16, force_new_instance=True)

In [ ]:
station.add_component(ivvi)

### small addition to VNA, to be able to set electrical delay via software

These functions are redundant from qcodes 0.9.0

In [ ]:
station.vna.S21.add_parameter(name='electrical_delay', 
                              get_cmd='SENS1:CORR:EDEL2:TIME?',
                              set_cmd='SENS1:CORR:EDEL2:TIME {}',
                              get_parser=float,
                              unit = 's')

In [ ]:
station.vna.S21.add_function('set_electrical_delay_auto',
                            call_cmd='SENS1:CORR:EDEL:AUTO ONCE')

## Generate database on drive D:/Data/ and name your sample

In [ ]:
sample_name = '191106_35nm_4p2_d2'
create_local_dbase_in(folder_name='pTdetection/sample1', db_name='%s.db' %(sample_name))

In [ ]:
qc.config['core']['db_location']

# Define pysweep measurement functions

In [ ]:
pysweep.STATION = station

In [ ]:
# Function called before beginning of measurement loop.
def init_measurement(d):
    station.vna.S21.avg(1)
    
# Function called after the end of the measurement.
def end_measurement(d):
    pass

In [ ]:
@MakeMeasurementFunction([pysweep.DataParameter('frequency','Hz', 'array', True),
                          pysweep.DataParameter('amplitude', '', 'array'), 
                          pysweep.DataParameter('phase', 'rad', 'array')])
def return_vna_trace(d):
    freqs = np.linspace(station.vna.S21.start(),station.vna.S21.stop(), station.vna.S21.npts())
    if not station.vna.rf_power():
        station.vna.rf_on()
    vna_data = station.vna.S21.trace_mag_phase()
    return [freqs, vna_data[0], vna_data[1]]

# Single VNA trace

In [ ]:
# Function called before beginning of measurement loop.
def init_measurement(d):
    station.vna.S21.avg(3)
    station.vna.S21.power(-20)
    station.vna.S21.bandwidth(1000)
    
    
    vna = qc.Station.default.vna
    trace = getattr(vna.channels, 'S21')
    
    trace.center(4.912e9)
    trace.span(40e6)
    trace.npts(2019)
    
    station.vna.S21.set_electrical_delay_auto()
    
    
# Function called after the end of the measurement.
def end_measurement(d):
    pass

In [ ]:
exp=select_experiment('VNA_trace', 'NbTiN_4p9')
meas = Measurement(exp, station)

r = pysweep.sweep(init_measurement, end_measurement, return_vna_trace,
                 databackend = pysweep.databackends.qcodes.DataBackend(meas))

In [ ]:
qc.dataset.plotting.plot_dataset(r.datasaver.dataset)

# Gate sweep

In [ ]:
# Function called before beginning of measurement loop.
def init_measurement(d):
    station.vna.S21.avg(3)
    station.vna.S21.power(-20)
    
    vna = qc.Station.default.vna
    trace = getattr(vna.channels, 'S21')
    
    trace.center(10.0e9)
    trace.span(5.0e9)
    trace.npts(8000)
    
    
# Function called after the end of the measurement.
def end_measurement(d):
    pass

In [ ]:
ivvi.set_dacs_zero()

In [ ]:
np.arange(-20, -15, 5)

In [ ]:
exp=select_experiment('VNA_vs_matrix_1_3', 'full_spectrum_upper')
meas = Measurement(exp, station)

r = pysweep.sweep(init_measurement, end_measurement, return_vna_trace, 
                  pysweep.sweep_object(ivvi.dac1, np.arange(-2000., 2050., 100)),
                  databackend = pysweep.databackends.qcodes.DataBackend(meas))

# Power sweep

In [ ]:
# Function called before beginning of measurement loop.
def init_measurement(d):
    station.vna.S21.avg(1)
    station.vna.S21.power(-20)
    station.vna.S21
    
    vna = qc.Station.default.vna
    trace = getattr(vna.channels, 'S21')
    
    trace.center(4.912e9)
    trace.span(50e6)
    trace.npts(2019)
    
    
# Function called after the end of the measurement.
def end_measurement(d):
    pass

In [ ]:
exp=select_experiment('VNA_vs_pwr', 'NbTiN_4p9')
meas = Measurement(exp, station)

r = pysweep.sweep(init_measurement, end_measurement, return_vna_trace, 
                  pysweep.sweep_object(station.vna.S21.power, np.arange(-20, -15, 1)),
                  databackend = pysweep.databackends.qcodes.DataBackend(meas))

# Magnetic field sweep

## x, y, z setup

In [ ]:
station.mgnt.field_ramp_rate(FieldVector(x=3e-3, y=3e-3, z=3e-3))

In [ ]:
print(station.mgnt.x_measured())
print(station.mgnt.y_measured())
print(station.mgnt.z_measured())

In [ ]:
station.mgnt.x_target(0.0e-3)
station.mgnt.y_target(0.0e-3)
station.mgnt.z_target(0.1e-3)

station.mgnt.ramp(mode='safe')

In [ ]:
print(station.mgnt.r_measured())
print(station.mgnt.phi_measured())
print(station.mgnt.theta_measured())

In [ ]:
station.mgnt.r_target(10.0e-3)
station.mgnt.phi_target(-180)
station.mgnt.theta_target(3)

station.mgnt.ramp(mode='safe')

## phi sweep

### Damaz test bench

In [ ]:
# Track the magnet movement 
@MakeMeasurementFunction([pysweep.DataParameter(name='x', unit='T'), 
                          pysweep.DataParameter(name='y', unit='T'),
                          pysweep.DataParameter(name='z', unit='T')])

def measure_magnet_orientation(d):
    x_meas = station.mgnt.x_measured()
    y_meas = station.mgnt.y_measured()
    z_meas = station.mgnt.z_measured()
    return [x_meas, y_meas, z_meas] 

In [ ]:
def sweep_phi(r, theta, points):
    @MakeMeasurementFunction([])
    def point_function(d):
        return points, []
    
    @MakeMeasurementFunction([])
    def set_function(phi, d):
        # Here we use the ISO 80000-2:2009 physics convention for the (r, theta, phi) <--> (x, y, z) definition. 
        # Note that r is the radial distance, theta the inclination and phi the azimuth (in-plane) angle. 
        # For uniqueness, we restrict the parameter choice to r>=0, 0<= theta <= pi and 0<= phi <= 2pi. 
        # The units are: r in T, theta in degrees, phi in degrees. 
        
        assert 1.>r>0., 'The radial distance must be lager than 0. For safty, r<1T Change setting!' 
        assert 0.<=theta<=180., 'The inclination angle must be equal or lager than 0 and smaller or equal than 180. Change setting!'  
        assert 0.<=phi<=2*180., 'The azimuth angle must be equal or lager than 0 and smaller or equal than 360. Change setting!'  
     
        station.mgnt.field_ramp_rate(FieldVector(x=3e-3, y=3e-3, z=3e-3))
        
        x = r*np.sin(np.radians(theta))*np.cos(np.radians(phi))
        y = r*np.sin(np.radians(theta))*np.sin(np.radians(phi))
        z = r*np.cos(np.radians(theta))        
        
        station.mgnt.x_target(x)
        station.mgnt.y_target(y)
        station.mgnt.z_target(z)

        station.mgnt.ramp(mode='safe')
        
        return []
    
    return SweepObject(set_function = set_function, unit = "degrees", label = "phi_var", point_function = point_function, dataparameter=None )

In [ ]:
# Function called before beginning of measurement loop.
def init_measurement(d):
    station.vna.S21.avg(1)
    station.vna.S21.power(-15)
    station.vna.S21.bandwidth(100)    
    station.vna.S21.set_electrical_delay_auto()
    
    vna = qc.Station.default.vna
    trace = getattr(vna.channels, 'S21')
    
    trace.center(4.912e9)
    trace.span(40e6)
    trace.npts(2019)
    
    station.mgnt.field_ramp_rate(FieldVector(x=3e-3, y=3e-3, z=3e-3))
   
    
# Function called after the end of the measurement.
def end_measurement(d):
    pass

In [ ]:
exp=select_experiment('VNA_vs_mgnt-phi', 'NbTiN_4p9')
meas = Measurement(exp, station)

r = pysweep.sweep(init_measurement, end_measurement, measure_magnet_orientation + return_vna_trace, 
                  sweep_phi(r = 20.0e-3, theta = 5, points = np.arange(0,360, 5)),
                  databackend = pysweep.databackends.qcodes.DataBackend(meas))

In [ ]:
qc.dataset.plotting.plot_dataset(r.datasaver.dataset)

## r sweep

In [ ]:
# Function called before beginning of measurement loop.
def init_measurement(d):
    station.vna.S21.avg(1)
    station.vna.S21.power(-15)
    station.vna.S21.bandwidth(100)    
    station.vna.S21.set_electrical_delay_auto()
    
    vna = qc.Station.default.vna
    trace = getattr(vna.channels, 'S21')
    
    trace.center(4.912e9)
    trace.span(40e6)
    trace.npts(2019)
    
    station.mgnt.field_ramp_rate(FieldVector(x=3e-3, y=3e-3, z=3e-3))
   
    
# Function called after the end of the measurement.
def end_measurement(d):
    pass

In [ ]:
exp=select_experiment('VNA_vs_mgnt-r', 'NbTiN_4p9')
meas = Measurement(exp, station)

r = pysweep.sweep(init_measurement, end_measurement, return_vna_trace, 
                  pysweep.sweep_object(station.mgnt.r_ramp, np.arange(station.mgnt.r_measured(), 250.0e-3, 5.0e-3)),
                  databackend = pysweep.databackends.qcodes.DataBackend(meas))

In [ ]:
qc.dataset.plotting.plot_dataset(r.datasaver.dataset)

## theta sweep

In [ ]:
# Function called before beginning of measurement loop.
def init_measurement(d):
    station.vna.S21.avg(3)
    station.vna.S21.power(-20)
    station.vna.S21.bandwidth(1000)    
    
    vna = qc.Station.default.vna
    trace = getattr(vna.channels, 'S21')
    
    trace.center(4.912e9)
    trace.span(40e6)
    trace.npts(2019)
    
    station.mgnt.field_ramp_rate(FieldVector(x=3e-3, y=3e-3, z=3e-3))
    
    station.vna.S21.set_electrical_delay_auto('Thanks to Lukas the GREAT')
    
    
# Function called after the end of the measurement.
def end_measurement(d):
    pass

In [ ]:
exp=select_experiment('VNA_vs_mgnt-theta', 'NbTiN_4p9')
meas = Measurement(exp, station)

r = pysweep.sweep(init_measurement, end_measurement, return_vna_trace, 
                  pysweep.sweep_object(station.mgnt.theta_ramp, np.arange(-5, 5., 1.)),
                  databackend = pysweep.databackends.qcodes.DataBackend(meas))

In [ ]:
qc.dataset.plotting.plot_dataset(r.datasaver.dataset)

# Sandbox

In [ ]:
def measure_vna_trace(station, sample_name=None, exp_name=None, fit_resonator=False, chan='S21'):
    """
    Measure a single VNA trace as currently set up.
    Default measurement is S21.
    """
    def get_freq_ax():
        return np.linspace(trace.start(), trace.stop(). trace.npts())
    
    if exp_name is None:
        exp_name = 'VNA_trace'

    exp = qc.load_or_create_experiment(experiment_name=exp_name, sample_name=sample_name)
        
    trace = getattr(station.vna.channels, chan)    
    freqs = qc.ManualParameter('frequency', unit='Hz', set=None, initial_value=np.linspace(t.start(), t.stop(), t.npts()))
    vna_data = qc.MultiParameter('vna_data', names=('amplitude', 'phase'), units=('', 'rad'), shapes=((),()), get=trace.trace_mag_phase())
    
    meas = qc.Measurement(exp=exp_name, station=station)
    meas.register_parameter(freqs)
    meas.register_parameter(vna_data, setpoints=(freqs, ))
    
    with meas.run() as datasaver:
        datasaver.add_result((freqs, freqs()),
                             (vna_data, trace.mag_phase()))